In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import torchvision
import torchvision.transforms as transforms


In [8]:
#figuring out mean and std


# import torch 
# from torchvision import datasets, transforms

# mean = 0.0
# std = 0.0
# for images, _ in loader:
#     batch_samples = images.size(0) 
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)

# mean /= len(loader.dataset)
# std /= len(loader.dataset)

# print(mean)
# print(std)

In [10]:
import torch
from torchvision import transforms, datasets
import cv2
from PIL import ImageFile 
import pathlib
import random 
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler 
import os

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'

use_gpu = torch.cuda.is_available()
print("GPU Available: {}".format(use_gpu))

# input_folder = 'CervicalCancer/Additional /Type_2'
# input_folder = 'CervicalCancer/CC_smallDataset'
input_folder = 'CervicalCancer/Train'
test_folder = 'CervicalCancer/Test'



#lets check how many files are there in the path we defined
# count = 0
# for path in pathlib.Path(input_folder).iterdir():
#     if path.is_file():
#         count += 1
# print(count)



transform = transforms.Compose( [transforms.Resize((256,256)),
#                                  transforms.CenterCrop(128),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5148, 0.4237, 0.4607), (0.1985, 0.1767, 0.1888))])


# change from CIFAR: adding .ImageFolder
# now, we can load
trainset = datasets.ImageFolder(root=input_folder,
                                           transform=transform)
# testset = datasets.ImageFolder(root=test_folder,transform=transform)


# split the validation 
valid_size = 0.15
num_train = len(trainset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

# if we want to shuffle:
shuffle = True
if shuffle:
    np.random.seed(random.randint(1,10))
    np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)



train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,sampler=train_sampler,
                                            num_workers=2)


valid_loader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=valid_sampler,
                                            num_workers=2)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=32,num_workers=2)


# lets see what we have in the train loader
# for i, data in enumerate(train_loader):
#     # get the inputs; data is a list of [inputs, labels]
#     print(data)
#     inputs, labels = data
#     print(f'this is input: {inputs},this is labels : {labels}')


#check if the train, valid works
# count = 0                                       
# for i, data in enumerate(valid_loader):
#     count +=1
#     inputs, labels = data
# print(f'this is the number of images in validation:{count}')


GPU Available: True


In [12]:
# Let's define a network

#this is densenet network
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)

# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)



#the followng is the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.net = nn.Sequential()
        self.net.add_module('cv1', nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0, dilation=1))
        self.net.add_module('rl1', nn.ReLU()) # activation functions don't change the size
        self.net.add_module('bn1', nn.BatchNorm2d(num_features = 32))
        self.net.add_module('cv2', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0, dilation=1))
        self.net.add_module('rl2', nn.ReLU())
        self.net.add_module('bn2', nn.BatchNorm2d(num_features = 32))
        self.net.add_module('mp1', nn.MaxPool2d(kernel_size=2, stride=None, padding=0, dilation=1)) # choose 1 pixel from 2x2 matrix (spatial_dim/kernel_size) (reduce the size)
        self.net.add_module('dp1', nn.Dropout2d(p=0.25))
        self.net.add_module('fl1', nn.Flatten())
        self.net.add_module('fc1', nn.Linear(in_features=1016064, out_features=128)) # 64 * spatial_dim_1 * spatial_dim_2 = (in_features)
        self.net.add_module('rl3', nn.ReLU())                                     # spatial_dim_1 = 28; spatial_dim_2 = 28
        self.net.add_module('dp2', nn.Dropout(p=0.5))
        self.net.add_module('fc2', nn.Linear(in_features=128, out_features=3))
        self.net.add_module('sm1', nn.LogSoftmax(dim=1))

    def forward(self, x):
        return self.net(x)

# chossing the model which in we want to feed the data 

# model = Net().to(device) ##this is CNN
model = model.to(device)

# Let's define an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001) # adjust the learning rate 

# Let's define a Loss function
lossfun = nn.NLLLoss()  # Use nn.CrossEntropyLoss with softmax


Using cache found in /home/ec2-user/.cache/torch/hub/pytorch_vision_v0.6.0


In [13]:
#MELTEM: output file for results
logfile = open('./logfile_{}.txt'.format("CC_Resnet12_20epochs"), 'a')

#lets train the model

# # Training
def train(epoch,loader,net):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = lossfun(outputs, targets)
        loss.backward()
        optimizer.step()  

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()   

        # MELTEM:removed
        #progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

        #MELTEM:added
        msg = 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total)
#         progress_bar(batch_idx, len(trainloader), msg)
        logfile.write('Train epoch {}. {}\n'.format(epoch, msg))
    
    return train_loss, 100.*correct/total



In [14]:
## lets implement validation check 

best_acc = 0


def valid(epoch,valid_loader,net):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valid_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = lossfun(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            #MELTEM:removed
            #progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

            #MELTEM:added
            msg = 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total)
#             progress_bar(batch_idx, len(testloader), msg)
            logfile.write('Test epoch {}. {}\n'.format(epoch, msg))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),   
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')   
        best_acc = acc
        
    return test_loss, acc




In [19]:
# for epoch in range(start_epoch, start_epoch+max_nbr_epoch):
t_loss_a = []
v_loss_a = []
temp = []
count = 0 
for epoch in range(1,2):
    temp.append(count)
    count+=1
    t_loss, t_accuracy = train(epoch,train_loader,model)
    v_loss, v_accuracy = valid(epoch,valid_loader,model)
    t_loss_a.append(t_loss)
    v_loss_a.append(v_loss)
    
plt.plot(temp, t_loss_a, label='train')
plt.plot(temp, v_loss_a, label='valid')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()
logfile.close()


Epoch: 1


ValueError: I/O operation on closed file.

In [65]:
def test(model, testloader):

  # evaluating
  model.eval()

  test_accuracy = 0

  with torch.no_grad():
    for data, target in testloader:

      if use_gpu:
        data = data.cuda()
        target = target.cuda()

      output = model(data)

      test_accuracy += (output.argmax(-1) == target).float().sum().item()
  
  test_accuracy /= len(testloader.dataset)
  print("Test set accuracy: {}".format(test_accuracy ))
  
  return test_accuracy 

In [67]:
test(model, test_loader)

NameError: name 'test_loader' is not defined

In [37]:
#attempting confusion matrix 

print(len(trainset))
print(len(trainset.targets))

#getting the prediction on a tensor
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)
        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(trainset, batch_size=10000)
    train_preds = get_all_preds(model,prediction_loader)

preds_correct = get_num_correct(train_preds, trainset.targets)

# print('total correct:', preds_correct)
# print('accuracy:', preds_correct / len(trainset))
    

1483
1483


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

